In [196]:
!pip install tensorboardX

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time

from torchvision import datasets, transforms
from tensorboardX import SummaryWriter

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Simple NN. You can change this if you want.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 50)
        self.fc2 = nn.Linear(50,50)
        self.fc3 = nn.Linear(50,50)
        self.fc4 = nn.Linear(50,10)

    def forward(self, x):
        x = x.view((-1, 28*28))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

class Normalize(nn.Module):
    def forward(self, x):
        return (x - 0.1307)/0.3081

# Add the data normalization as a first "layer" to the network
# this allows us to search for adverserial examples to the real image, rather than
# to the normalized image
model = nn.Sequential(Normalize(), Net())

model = model.to(device)
model.train()

Sequential(
  (0): Normalize()
  (1): Net(
    (fc1): Linear(in_features=784, out_features=50, bias=True)
    (fc2): Linear(in_features=50, out_features=50, bias=True)
    (fc3): Linear(in_features=50, out_features=50, bias=True)
    (fc4): Linear(in_features=50, out_features=10, bias=True)
  )
)

In [197]:
def train_model(model, num_epochs, enable_defense=True):
    learning_rate = 0.0001

    opt = optim.Adam(params=model.parameters(), lr=learning_rate)

    ce_loss = torch.nn.CrossEntropyLoss()

    tot_steps = 0

    for epoch in range(1,num_epochs+1):
        t1 = time.time()
        for batch_idx, (x_batch, y_batch) in enumerate(train_loader):

            if enable_defense:
                pass
                ###############################################
                # Fill code here to do adversarial training
                # You may find it useful to switch to 'eval' model while generating the attack
                # and switch back again to 'train' mode once the attack is generated.
                ###############################################

            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            tot_steps += 1
            opt.zero_grad()
            out = model(x_batch)
            batch_loss = ce_loss(out, y_batch)
            batch_loss.backward()
            opt.step()

        tot_test, tot_acc = 0.0, 0.0
        for batch_idx, (x_batch, y_batch) in enumerate(test_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            out = model(x_batch)
            pred = torch.max(out, dim=1)[1]
            acc = pred.eq(y_batch).sum().item()
            tot_acc += acc
            tot_test += x_batch.size()[0]
        t2 = time.time()

        print('Epoch %d: Accuracy %.5lf [%.2lf seconds]' % (epoch, tot_acc/tot_test, t2-t1))

In [198]:
train_model(model, 15, False)

Epoch 1: Accuracy 0.90000 [8.90 seconds]
Epoch 2: Accuracy 0.91750 [8.74 seconds]
Epoch 3: Accuracy 0.92610 [8.64 seconds]
Epoch 4: Accuracy 0.93030 [8.64 seconds]
Epoch 5: Accuracy 0.93620 [8.73 seconds]
Epoch 6: Accuracy 0.94060 [8.67 seconds]
Epoch 7: Accuracy 0.94540 [8.69 seconds]
Epoch 8: Accuracy 0.94860 [8.74 seconds]
Epoch 9: Accuracy 0.95250 [8.79 seconds]
Epoch 10: Accuracy 0.95440 [8.64 seconds]
Epoch 11: Accuracy 0.95570 [8.70 seconds]
Epoch 12: Accuracy 0.95880 [8.87 seconds]
Epoch 13: Accuracy 0.96160 [8.69 seconds]
Epoch 14: Accuracy 0.96200 [8.68 seconds]
Epoch 15: Accuracy 0.96540 [8.69 seconds]


In [199]:
def ibp(model, low, high):
    nw0, nb0, nw1, nb1, nw2, nb2, nw3, nb3 = model.parameters()
    low= low.view((-1,784,1))
    high = high.view((-1,784,1))
    vrelu = torch.nn.functional.relu
    
    low1 = vrelu(torch.matmul(torch.max(nw0, torch.tensor(0.0)), low) + torch.matmul(torch.min(nw0, torch.tensor(0.0)), high) + nb0.reshape(-1,1))
    high1 = vrelu(torch.matmul(torch.max(nw0, torch.tensor(0.0)), high) + torch.matmul(torch.min(nw0, torch.tensor(0.0)), low) + nb0.reshape(-1,1))
    low2 = vrelu(torch.matmul(torch.max(nw1, torch.tensor(0.0)), low1) + torch.matmul(torch.min(nw1, torch.tensor(0.0)), high1) + nb1.reshape(-1,1))
    high2 = vrelu(torch.matmul(torch.max(nw1, torch.tensor(0.0)), high1) + torch.matmul(torch.min(nw1, torch.tensor(0.0)), low1) + nb1.reshape(-1,1))
    low3 = vrelu(torch.matmul(torch.max(nw2, torch.tensor(0.0)), low2) + torch.matmul(torch.min(nw2, torch.tensor(0.0)), high2) + nb2.reshape(-1,1))
    high3 = vrelu(torch.matmul(torch.max(nw2, torch.tensor(0.0)), high2) + torch.matmul(torch.min(nw2, torch.tensor(0.0)), low2) + nb2.reshape(-1,1))
    out_low  = torch.matmul(torch.max(nw3, torch.tensor(0.0)), low3) + torch.matmul(torch.min(nw3, torch.tensor(0.0)), high3) + nb3.reshape(-1,1)
    out_high  = torch.matmul(torch.max(nw3, torch.tensor(0.0)), high3) + torch.matmul(torch.min(nw3, torch.tensor(0.0)), low3) + nb3.reshape(-1,1)

    return out_low, out_high

# took help from github codebase mentioned by professor: https://github.com/google-deepmind/interval-bound-propagation/tree/217a14d12686e08ebb5cfea1f2748cce58a55913/examples

In [200]:
num_classes=10
def testModel(model, eps, enable_defense=True):
    tot_test, tot_acc = 0.0, 0.0
    for batch_idx, (x_batch, y_batch) in enumerate(test_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        out_low, out_high = ibp(model, torch.maximum(x_batch - eps, torch.tensor(0)), torch.minimum(x_batch + eps, torch.tensor(1)))
#         print(out_low[0])
        for idx, target_class in enumerate(y_batch):
            ans = True
            for i in range(num_classes):
                if i != target_class and out_low[idx][target_class] < out_high[idx][i]:
                    ans = False
            if ans:
                tot_acc = tot_acc + 1
            
        tot_test += x_batch.size()[0]
    print('Epoch %d: Accuracy %.5lf Eps: %.2lf' % (0, tot_acc/tot_test, eps))

In [201]:
epsilons = torch.linspace(0.01, 0.1, 10)

for epsilon in epsilons:
    testModel(model, epsilon)

Epoch 0: Accuracy 0.00000 Eps: 0.01
Epoch 0: Accuracy 0.00000 Eps: 0.02
Epoch 0: Accuracy 0.00000 Eps: 0.03
Epoch 0: Accuracy 0.00000 Eps: 0.04
Epoch 0: Accuracy 0.00000 Eps: 0.05
Epoch 0: Accuracy 0.00000 Eps: 0.06
Epoch 0: Accuracy 0.00000 Eps: 0.07
Epoch 0: Accuracy 0.00000 Eps: 0.08
Epoch 0: Accuracy 0.00000 Eps: 0.09
Epoch 0: Accuracy 0.00000 Eps: 0.10


In [202]:
def train_model_(model, num_epochs, eps):
    learning_rate = 0.001

    opt = optim.Adam(params=model.parameters(), lr=learning_rate)

    ce_loss = torch.nn.CrossEntropyLoss()

    tot_steps = 0

    for epoch in range(1,num_epochs+1):
        t1 = time.time()
        for batch_idx, (x_batch, y_batch) in enumerate(train_loader):

#             if enable_defense:
#                 pass
                ###############################################
                # Fill code here to do adversarial training
                # You may find it useful to switch to 'eval' model while generating the attack
                # and switch back again to 'train' mode once the attack is generated.
                ###############################################

            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            tot_steps += 1
            opt.zero_grad()
            out_low, out_high = ibp(model, torch.maximum(x_batch - eps, torch.tensor(0)), torch.minimum(x_batch + eps, torch.tensor(1)))
            one_hot = torch.eye(10)[y_batch,:]
            batch_loss = ce_loss(out_low[:,:,0]*one_hot + out_high[:,:,0]*(1-one_hot), y_batch)
            batch_loss.backward()
            opt.step()

        tot_test, tot_acc = 0.0, 0.0
        for batch_idx, (x_batch, y_batch) in enumerate(test_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            out = model(x_batch)
            pred = torch.max(out, dim=1)[1]
            acc = pred.eq(y_batch).sum().item()
            tot_acc += acc
            tot_test += x_batch.size()[0]
        t2 = time.time()

        print('Epoch %d: Accuracy %.5lf [%.2lf seconds]' % (epoch, tot_acc/tot_test, t2-t1))

In [203]:
train_model_(model, 20, 0.01)

Epoch 1: Accuracy 0.86510 [13.31 seconds]
Epoch 2: Accuracy 0.88630 [13.04 seconds]
Epoch 3: Accuracy 0.88350 [13.11 seconds]
Epoch 4: Accuracy 0.88710 [12.78 seconds]
Epoch 5: Accuracy 0.88760 [12.92 seconds]
Epoch 6: Accuracy 0.88950 [12.69 seconds]
Epoch 7: Accuracy 0.88650 [12.79 seconds]
Epoch 8: Accuracy 0.87660 [12.82 seconds]
Epoch 9: Accuracy 0.88040 [12.66 seconds]
Epoch 10: Accuracy 0.88260 [12.67 seconds]
Epoch 11: Accuracy 0.88400 [12.75 seconds]
Epoch 12: Accuracy 0.87850 [12.70 seconds]
Epoch 13: Accuracy 0.88120 [12.54 seconds]
Epoch 14: Accuracy 0.87640 [12.62 seconds]
Epoch 15: Accuracy 0.88300 [12.98 seconds]
Epoch 16: Accuracy 0.87220 [12.81 seconds]
Epoch 17: Accuracy 0.87330 [12.87 seconds]
Epoch 18: Accuracy 0.88300 [12.64 seconds]
Epoch 19: Accuracy 0.87760 [12.77 seconds]
Epoch 20: Accuracy 0.87170 [12.66 seconds]


In [204]:
epsilons = torch.linspace(0.01, 0.1, 10)

for epsilon in epsilons:
    testModel(model, epsilon)

Epoch 0: Accuracy 0.93400 Eps: 0.01
Epoch 0: Accuracy 0.89940 Eps: 0.02
Epoch 0: Accuracy 0.84230 Eps: 0.03
Epoch 0: Accuracy 0.74500 Eps: 0.04
Epoch 0: Accuracy 0.59380 Eps: 0.05
Epoch 0: Accuracy 0.42540 Eps: 0.06
Epoch 0: Accuracy 0.28010 Eps: 0.07
Epoch 0: Accuracy 0.17710 Eps: 0.08
Epoch 0: Accuracy 0.11330 Eps: 0.09
Epoch 0: Accuracy 0.07290 Eps: 0.10


In [205]:
model = nn.Sequential(Normalize(), Net())

model = model.to(device)
model.train()

Sequential(
  (0): Normalize()
  (1): Net(
    (fc1): Linear(in_features=784, out_features=50, bias=True)
    (fc2): Linear(in_features=50, out_features=50, bias=True)
    (fc3): Linear(in_features=50, out_features=50, bias=True)
    (fc4): Linear(in_features=50, out_features=10, bias=True)
  )
)

In [206]:
train_model(model, 15, False)

Epoch 1: Accuracy 0.90270 [8.65 seconds]
Epoch 2: Accuracy 0.91770 [8.71 seconds]
Epoch 3: Accuracy 0.92810 [8.76 seconds]
Epoch 4: Accuracy 0.93370 [8.94 seconds]
Epoch 5: Accuracy 0.93930 [8.87 seconds]
Epoch 6: Accuracy 0.94160 [8.75 seconds]
Epoch 7: Accuracy 0.94660 [8.95 seconds]
Epoch 8: Accuracy 0.94940 [8.71 seconds]
Epoch 9: Accuracy 0.95190 [8.85 seconds]
Epoch 10: Accuracy 0.95440 [8.69 seconds]
Epoch 11: Accuracy 0.95770 [8.96 seconds]
Epoch 12: Accuracy 0.95990 [8.85 seconds]
Epoch 13: Accuracy 0.95900 [8.78 seconds]
Epoch 14: Accuracy 0.96100 [8.86 seconds]
Epoch 15: Accuracy 0.96340 [8.73 seconds]
